In [1]:
cd ../

/Users/ngriffiths/Deep_Learning_Name_Creation


In [87]:
import joblib
import torch
from typing import *
import numpy as np
import torch.nn as nn

In [69]:
names = list(joblib.load('data/names.pkl')['names'])

In [70]:
chars = set([c for word in names for c in word])

In [71]:
chars.add('<EOS>')
chars.add('!')
chars.add('unk')

In [72]:
ix_to_char = {i: c for i, c in enumerate(chars)}
char_to_ix = {c: i for i, c in ix_to_char.items()}

In [73]:
def name_to_ix(name: str,
                   return_y: bool=False
                  ) -> Union[torch.tensor, Optional[torch.tensor]]:
    #x = torch.tensor([char_to_ix[n] for n in name[:-1]])
    x = [char_to_ix[n] for n in name[:-1]]
    y = None
    if return_y:
        y = [char_to_ix[n] for n in name[1:]]
    return x, y

In [74]:
x_test, y_test = name_to_ix('nelson', True)

In [75]:
torch.tensor([x_test, x_test])

tensor([[20,  3, 48, 13, 30],
        [20,  3, 48, 13, 30]])

In [85]:
def build_batch(names: List[str], return_y:bool=False):
    x_batch = list()
    y_batch = list()
    
    max_chars = np.max([len(x) for x in names])
    padded_names = [n.ljust(max_chars, '!') for n in names]
    
    
    for name in padded_names:
        x_name, y_name = name_to_ix(name, return_y)
        x_batch.append(x_name)
        y_batch.append(y_name)
    x_batch = torch.tensor(x_batch)
    if return_y:
        y_batch = torch.tensor(y_batch)
        
    return x_batch, y_batch

In [130]:
# Build basic RNN

class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, pad):
        super(RNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim, pad)
        self.rnn = nn.RNN(embed_dim, 20, 2, dropout=.2)
        self.fc = nn.Linear(20, vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        x = self.embed(x)
        x, h_n = self.rnn(x)
        x = self.fc(x)
        return self.softmax(x)
        

In [131]:
model = RNN(len(chars), 20, char_to_ix['!'])

In [135]:
import random
name_sample = random.sample(names, 1000) 

In [145]:
def train(names: List[str], batch_size: int= 10):
    n_iters = len(names)//batch_size
    
    for i in range(n_iters):
        model.zero_grad()
        x_batch, y_batch = build_batch(names[batch_size*i: batch_size*(i+1)])
        
        preds = model(x_batch)
        print(preds.shape)

In [146]:
train(names[0:20], 10)

torch.Size([10, 7, 55])
torch.Size([10, 6, 55])
